In [83]:
# configuracion general y lectura de los datos
import plotly.express as px
import duckdb
import pandas as pd
import plotly.graph_objects as go
import numpy as np

with duckdb.connect("../legoDB/bbdd/legoPRO.duckdb") as con:
    colors_by_year = con.table('main.colors_by_year').df()
    colors_and_groups = con.table('main.colors_and_groups').df()
    themes_by_year = con.table('main.themes_by_year_graph').df()
colors_only_colors = colors_by_year.query('agg_type == "color"')
df1 = pd.merge(colors_only_colors,colors_and_groups)
df1.sort_values(['color_group_id', 'quantity'], ascending=[False, False], inplace=True)
df2 = themes_by_year

# Escala
scale=100

# Mapeo de los colores por nombre y hex
color_map = dict(zip(df1['color_name'], '#'+df1['color_rgb']))

# Normalizar los valores de theta para que ocupen 270 grados
theta_offset = 90  # Empezar en 90 grados
theta_max = 360    # Fin en 360 grados

# Definir el tamaño mínimo y máximo deseado para los marcadores
min_marker_size = scale/20
max_marker_size = scale/2


In [84]:
## Preparar el dataset de las bolitas de colores
# Asignar colores a los datos usando el color_map
df1['color'] = df1['color_name'].map(color_map)

# Normalizar los tamaños de los marcadores al rango deseado
size_normalized = np.interp(df1['quantity'], (df1['quantity'].min(), df1['quantity'].max()), (min_marker_size, max_marker_size))

# Normalizar los valores de theta para que ocupen 270 grados
df1['theta'] = -3.6*df1['year']+7016.4

# Crear la figura
fig = go.Figure()

# Añadir la traza de dispersión polar
temp_output = fig.add_trace(go.Scatterpolar(
                r=df1['color_name'],
                theta=df1['theta'],
                mode='markers',
                marker=dict(
                    size=size_normalized,
                    color=df1['color'],
                    sizemode='diameter'
                    ),
                text=df1['year'],
                hoverinfo='text'
                )
            )

In [85]:
# DF2 Preparar el dataset de los temas
# Parámetros para escalar y desplazar el eje r en los temas
# Número de temas diferentes
n_themes = df2['theme_name'].nunique()
scale_factor = 7  # Escala máxima para r
r_offset = scale_factor / n_themes  # Desplazamiento para cada serie

# Rellenar donde faltan años
# Ver cuántos años hay
years = df2['year'].unique()
df2.sort_values(['color_group_id'], ascending=[False], inplace=True)

In [86]:

# Genero del diccionario vacío de los valores de los años que faltan
missing_rows = []

# Averiguo todos los temas que hay y voy iterando para rellenar todos los años que faltan en cada uno de los temas
for theme in df2['theme_name'].unique():
#for theme in ['System', 'Gear']:
    # Genero un dataset por cada uno de los temas
    df_theme = df2[df2['theme_name'] == theme]
    color_group_id = df_theme['color_group_id'].unique()[0]
    color_group_name = df_theme['color_group_name'].unique()[0]
    color_group_rgb = df_theme['color_group_rgb'].unique()[0]
    for year in years:
        if year not in df_theme['year'].values:
            missing_rows.append({'year': year,
                                 'n_sets': np.nan, 
                                 'total_parts': np.nan, 
                                 'theme_name': theme, 
                                 'color_group_id': color_group_id, 
                                 'color_group_name': color_group_name, 
                                 'color_group_rgb': color_group_rgb })


In [87]:

# Convertir la lista de filas faltantes en un DataFrame y concatenar con df2
if missing_rows:
    missing_df = pd.DataFrame(missing_rows)
    df2 = pd.concat([df2, missing_df], ignore_index=True)

df2 = df2.sort_values(by=['theme_name', 'year'])

# Normalizar los valores de theta para que ocupen 270 grados
df2['theta'] = -3.6*df2['year']+7016.4

# Asignar colores a los datos usando el color_map
df2['color'] = df2['color_group_name'].map(color_map)
df2['color'] = df2['color'].fillna('#000000')



In [88]:
custom_template = go.layout.Template(
    layout=go.Layout(
        title_font=dict(family="Chalet, sans-serif", size=24, color="#333333"),
        font=dict(family="Chalet, sans-serif", size=14, color="#333333"),
        paper_bgcolor="#FFFFFF",  # Fondo de la figura
        plot_bgcolor="#FFFFFF",  # Fondo de la gráfica
        polar=dict(
            radialaxis=dict(
                showgrid=False,
                gridcolor="#E1E1E1",
                showline=False,
                linecolor="#333333",
                ticks="",
                tickwidth=scale/50,
                tickcolor="#333333",
                showticklabels= False
            ),
            angularaxis=dict(
                showgrid=False,
                gridcolor="#E1E1E1",
                showline=False,
                linecolor="#333333",
                ticks="outside",
                tickwidth=scale/50,
                tickcolor="#333333",
                tickvals=df1['year'],
                ticktext=df1['year'],
                direction='counterclockwise'
            )
        ),
        showlegend=True,
        legend=dict(
            bgcolor="#FFFFFF"
        ),
        #margin=dict(l=60, r=20, t=40, b=60),
        width=10*scale,
        height=10*scale
    )
)
fig.update_layout(template=custom_template)
#fig.show()

In [89]:
lista = df2[['theme_name', 'color_group_id']].drop_duplicates()
lista.sort_values(['color_group_id'], ascending=[False], inplace=True)


In [90]:
#  Añadir una traza para cada tema
# for i, theme in enumerate([
#                            'Technic', 'Super Heroes DC', 'LEGO Ideas and CUUSOO', 
#                            'LEGO Art', 'DOTS', 'Unikitty!', 
#                            'Modulex', 
#                            'Star Wars', 'City', 'Creator',
#                            'Clikits', 
#                            'Universal Building Set', 'Make & Create', 'Duplo', 
#                            'Architecture', "Pharaoh's Quest",
#                            'Friends', 'Seasonal', 'System',
#                            'Fabuland', 'Minions', 'Primo'
#                            ]):
for i, theme in enumerate(lista['theme_name']):
    df_theme = df2[df2['theme_name'] == theme]  # Filtrar el dataframe para el tema actual
    # Escalar y desplazar los valores de r
    #r_values = df_theme['n_sets'] + i * r_offset
    r_values = df_theme['total_parts'] #+ i * r_offset

    color_value = df_theme['color'].iloc[0]
    fig.add_trace(go.Scatterpolar(
        r=r_values,        
        theta=df_theme['theta'],
        mode='lines',   
        line=dict(
            width=2, 
            dash='solid',
            color=color_value
            ),
        name=theme,             
        text=r_values, 
        hoverinfo='text',
        connectgaps=False
    ))
fig.show()